In [2]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
from sympy.abc import *
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
#%matplotlib notebook #incompatible with mpmath

Error importing optional module geopandas
Traceback (most recent call last):
  File "c:\Users\engin\anaconda3\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\engin\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\engin\anaconda3\lib\site-packages\geopandas\__init__.py", line 3, in <module>
    from geopandas.geoseries import GeoSeries  # noqa
  File "c:\Users\engin\anaconda3\lib\site-package

This function queries the International Atomic Energy Agency's Nuclear Data Services (https://nds.iaea.org/).

API: https://nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html 

Example Jupyter Notebook for automating data downloads: https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_notebook.html  

In [3]:
def lc_read_csv(url):
    '''
    Query the livechart service and return a pandas dataframe. 
    format: lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")
    '''
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

 This function takes a dataframe of parent nuclides and returns a list of dataframes of daughter nuclides

In [4]:
def get_daughter_df(parent_df):
    '''
    This function takes a dataframe of parent nuclides and returns a 
    list of dataframes of daughter nuclides
    '''
    daughter_nucleus= [str(int(parent_df['d_z'][row]) + int(parent_df['d_n'][row])) + 
                    ''.join(parent_df['d_symbol'][row]) for row in range(parent_df.shape[0])]
    daughter_df = [lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus[row].lower() 
                    + "&rad_types=bm") for row in range(parent_df.shape[0])]
    daughter_df = pd.concat(daughter_df)
    return daughter_df


It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0

In [5]:
#It appears that when queried for stable nuclides, livechart return a 0x1 dataframe with only the number 0
lc_read_csv(livechart + "fields=decay_rads&nuclides=" + "16O" + "&rad_types=bm")

,0


In [6]:
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=42ar&rad_types=bm")
df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,233,16,100,0,0,599,6,9.32,0.17,1U,367,3.6,18,24,Ar


In [7]:
df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,0+,32.9,,11,Y,1.038223e+09,3.471262e+07,...,100,,599,6,19,23,K,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06


In [8]:
float(df['mean_energy']) * float(df['intensity_beta']) / 100

233.0

In [9]:
daughter_nucleus = str(int(df['d_z']) + int(df['d_n'])) + ''.join(df['d_symbol'])
#make all letters lowercase 
daughter_nucleus.lower()

'42k'

In [10]:
daughter_df = lc_read_csv(livechart + "fields=decay_rads&nuclides=" + daughter_nucleus.lower() + "&rad_types=bm")
daughter_df.iloc[ : , : 15]

,mean_energy,unc_me,intensity_beta,unc_ib,daughter_level_energy,max_energy,unc_me.1,log_ft,unc_lf,transition_type,anti_nu_mean_energy,unc_ame,p_z,p_n,p_symbol
0,21.41,0.21,0.07,0.01,3445.4,79.80,0.70,4.9800,0.0700,A,58.60,0.60,19,23,K
1,415.41,0.2,0.05,0.01,2424.3,1100.90,0.40,9.0500,0.0900,1NU,686.70,0.60,19,23,K
2,702.95,0.2,0.34,0.03,1837.2,1688.00,0.40,9.9200,0.0400,1U,986.49,0.36,19,23,K
3,824.32,0.17,17.64,0.09,1524.6,2000.60,0.30,7.5501,0.0023,1NU,1177.16,0.25,19,23,K
4,1565.86,,81.90,0.09,0.0,3525.22,0.18,9.4768,0.0006,1U,1961.70,0.09,19,23,K


In [11]:
daughter_df.iloc[ : , 15: ]

,p_energy_shift,p_energy,unc_pe,jp,half_life,operator_hl,unc_hl,unit_hl,half_life_sec,unc_hls,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06
1,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06
2,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06
3,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06
4,,0,,2-,12.355,,7,h,44478,25.2,...,100,,3525.26,18,20,22,Ca,31-May-2016,JUN CHEN{+#} AND BALRAJ SINGH,2022-05-06


In [12]:
daughter_df = get_daughter_df(df)

In [13]:
third_gen_df = get_daughter_df(daughter_df)
third_gen_df

,0


#Battery Object
We will make a list of dataframes. Each element of the list will correspond to a step in the decay chain.
Each simulated battery will be a battery object. Every battery object has a composition attribute. 
This composition will be a dictionary where the keys are the nuclei and the values are the moles of each 
nucleus. The class will have a chain decay function that calculates how many moles of each nucleus there are at a given time.

#Derivation of chain decay function. This function will calculate the moles of each nucleus after a given time ($\tau$). The are $m_{t}$ moles of the mother nucleus at time $t$ and $\delta_{\tau}$ moles of the daughter nucleus at a time $\tau$. There is a probability $P_{x}$ that a given nucleus $x$ with an age of $t$ and an e-folding time of $\lambda_{x}$ has not yet decayed.
\begin{equation}
\begin{split}
\Large{ m_t = m_0e^{\Large\frac{t}{-\lambda_{m}}} } \\
\Large{ P_{x} = 1-e^{\Large\frac{t_{x}}{-\lambda_{x}}} }\\
\Large{ \delta_{\tau} = \int_{0}^{\tau} P_{\delta}d\delta_{new} }\\
\end{split}
\end{equation}
$\delta_{new}$ is the number of daughter nuclei that were created instantaneously at the time $t$. In these decay chains, the only source of new daughter nuclei is the decay of the mother nuclei so we have
\begin{equation}
\begin{split}
\Large{ \frac{d\delta_{new}}{dt} = -\frac{dm}{dt} = \frac{m_0e^{\Large\frac{t}{-\lambda_{m}}}}{\lambda_{m} } }\\
\Large{ d\delta_{new} = \frac{m_0e^{\Large\frac{t}{-\lambda_{m}}}}{\lambda_{m} } }dt \\
\Large{ \delta_{\tau} = \int_{0}^{\tau} \frac{m_0e^{\Large\frac{t}{-\lambda_{m}}}}{\lambda_{m} }(1-e^{\Large\frac{t}{-\lambda_{\delta}}}) dt}\\
\Large{ \delta_{\tau} = \frac{m_0}{\lambda_{m}}\int_{0}^{\tau} e^{\Large\frac{t}{-\lambda_{m}}}(1-e^{\Large\frac{t}{-\lambda_{\delta}}}) dt}\\
\Large{ \delta_{\tau} = \frac{m_0}{\lambda_{m}}(\int_{0}^{\tau} e^{\Large\frac{t}{-\lambda_{m}}}dt - \int_{0}^{\tau} e^{\Large (\frac{t}{-\lambda_{\delta}} + \frac{t}{-\lambda_{m}})} dt) }\\
\Large{ \delta_{\tau} = \frac{m_0}{\lambda_{m}}(\int_{0}^{\tau} e^{\Large\frac{t}{-\lambda_{m}}}dt - \int_{0}^{\tau} e^{\Large t(\frac{1}{-\lambda_{\delta}} + \frac{1}{-\lambda_{m}})} dt) }
\end{split}
\end{equation}
since 
\begin{equation}
\begin{split}
\Large\int_{0}^{\tau} e^{\Large kt}dt = \frac{e^{\Large kt}\mid_{0}^{\tau}}{k} = \frac{e^{\Large k\tau}-1}{k}\\
\end{split}
\end{equation}
then we have
\begin{equation}
\begin{split}
\Large \delta_{\tau} = \frac{m_0}{\lambda_m}(\lambda_m\lambda_\delta\frac{e^{\Large-\tau(\frac{1}{\lambda_m}+\frac{1}{\lambda_{\delta}})}-1}{\lambda_{\delta}+\lambda_{m}} - \lambda_m(e^{\Large-t\frac{1}{\lambda_m}}-1))\\
\Large\delta_{\tau} = m_0(\lambda_\delta\frac{e^{\Large-\tau(\frac{1}{\lambda_m}+\frac{1}{\lambda_{\delta}})}-1}{\lambda_{\delta}+\lambda_{m}} - e^{\Large-t\frac{1}{\lambda_m}} +1)
\end{split}
\end{equation}

In [18]:
delta_new, m_0, lambda_m, lambda_d = sym.symbols('\delta_{new} m_0 lambda_m lambda_delta', 
                positive = True, real = True)
delta_tau, tau = sym.symbols('delta_{tau} tau', positive = True, real = True)
lambda_t = sym.symbols('lambda')
#the probability a nucleus with a given e-folding time l after a time t
p = 1 - sym.exp(-1 * tau / lambda_d)
#the daughter nuclei being instantaneously created at a given time t
delta_new = m_0 * sym.exp(-1 * tau / lambda_m) / lambda_m
#daughter nuclei as a function of time
delta_tau = m_0 * (lambda_d * (sym.exp(-1 * tau * (1/lambda_d + 1/lambda_m)) - 1) / (lambda_m + lambda_d) - 
            sym.exp(-1 * tau / lambda_m) + 1)
delta_tau

m_0*(lambda_delta*(-1 + exp(-tau*(1/lambda_m + 1/lambda_delta)))/(lambda_delta + lambda_m) + 1 - exp(-tau/lambda_m))

In [26]:
attempt = sym.simplify(
    sym.integrate(delta_new * p, (tau, 0, t)))
attempt

m_0*(-lambda_delta*exp(t/lambda_delta) + lambda_delta - lambda_m*exp(t/lambda_delta) + lambda_m*exp(t*(lambda_delta + lambda_m)/(lambda_delta*lambda_m)))*exp(-t/lambda_m - t/lambda_delta)/(lambda_delta + lambda_m)

In [29]:
attempt = sym.integrate(delta_new * p, (tau, 0, t))
eval_attempt = sym.lambdify(([t, lambda_m, lambda_d, m_0]), attempt, 'sympy')
#specify the implementation of the function to be consistent with the input types
eval_attempt(sym.N(0), sym.N(1), sym.N(1), sym.N(1))

0

In [18]:
evaluate_delta_tau = sym.lambdify(([tau, lambda_m, lambda_d, m_0]), delta_tau)
evaluate_delta_tau(0, 1, 1, 1) #should be 0

0.0

In [19]:
evaluate_delta_tau(1, 32.9, 12 * units.h.to(units.year), 1) 

0.029896242480243584

In [30]:
p_t = 1 - sym.exp(-1 * t / lambda_t)
prob_not_decay = sym.lambdify(([t, lambda_t]), p_t)
def daughter_nucleus_abundance(t, lambda_d, parent_initial_mol, parent_nucleus_decay):
    m_0 = parent_initial_mol
    p = 1 - sym.exp(-1 * t / lambda_d)
    delta_tau = sym.integrate(p * parent_nucleus_decay, (t, 0, t))
    evaluate_delta_tau = sym.lambdify(([tau, lambda_m, lambda_d, m_0]), delta_tau, 'numpy')
    t, lambda_d, parent_initial_mol = np.float(t), np.float(lambda_d), np.float(parent_initial_mol)
    return evaluate_delta_tau(t, lambda_d, m_0, parent_nucleus_decay)



1.00000000000000

#General Case of daughter nucleus abundance
\begin{equation}
\begin{split}
\Large{ \delta_{\tau} = \int_{0}^{\tau} P_{\delta}d\delta_{new} }\\
\end{split}
\end{equation}
Again since the only source of new daughter nuclei is the decay of parent nuclei
\begin{equation}
\begin{split}
\Large \frac{d\delta_{new}}{dt} = -\frac{dm_t}{dt}\\
\Large d\delta_{new} = -\frac{dm_t}{dt}dt\\
\Large{ \delta_{\tau} = -\int_{0}^{\tau} P_{\delta}\frac{dm_t}{dt}dt }\\
\end{split}
\end{equation}
Assuming each individual daughter nucleus also has a measurable e-folding time $\lambda_{\delta}$ of the probability that it has not decayed
\begin{equation}
\begin{split}
\Large{ P_{\delta} = 1-e^{\Large\frac{t}{-\lambda_{\delta}}} }\\
\Large{ \delta_{\tau} = -\int_{0}^{\tau}(1-e^{\Large\frac{t}{-\lambda_{\delta}}})\frac{dm_t}{dt}dt }\\
\end{split}
\end{equation}
Assuming the parent nucleus has an abundance of the form
\begin{equation}
\begin{split}
\Large m_t = \int_0^{t}\alpha_tP_{m} dt \\
\Large{ \delta_{\tau} = -\int_{0}^{\tau}(1-e^{\Large\frac{t}{-\lambda_{\delta}}})\alpha_tP_{m} dt }\\
\end{split}
\end{equation}
Consider a decay chain represented by the sequence $n_{0}, n_{1}, n_{2}, ... n_{N}$. By induction we have for $i>0$
\begin{equation}
\begin{split}
\Large dn_{i} = -\frac{dn_{i-1}}{dt}dt\\
\Large n_{i} = \int_0^t P_{\Large n_i} dn_{i}\\
\Large n_{i} = -\int_0^t P_{\Large n_i} \frac{dn_{i-1}}{dt}dt\\
\end{split}
\end{equation}
Again assuming every nucleus for $i<N$ has a measurable e-folding time $\lambda_{n_{i}}$ of the probability that it has not decayed (we take $P_{\Large n_N} = 1$)
\begin{equation}
\begin{split}
\Large P_{\Large n_{i}} = 1-e^{\frac{\Large t}{\Large -\lambda_{\Large n_i}}} \\
\Large n_{i} = \int_0^t(e^{\frac{\Large t}{\Large -\lambda_{\Large n_i}}}-1)\frac{dn_{i-1}}{dt}dt\\
\end{split}
\end{equation}
Futhermore, 
\begin{equation}
\begin{split}
\Large \frac{dn_{i-1}}{dt} = \prod_{j = 0}^{i-1}P_{\Large n_{j}} =  \prod_{j = 0}^{i-1}(1-e^{\frac{\Large t}{\Large -\lambda_{\Large n_j}}}(-1^{j}))\\
\Large n_{i} = \int_0^t(e^{\frac{\Large t}{\Large -\lambda_{\Large n_i}}}-1)\prod_{j = 0}^{i-1}(1-e^{\frac{\Large t}{\Large -\lambda_{\Large n_j}}}(-1^{j})))dt\\
\Large n_{i} = \int_0^t\prod_{j = 0}^{i-1}(1-e^{\frac{\Large t}{\Large -\lambda_{\Large n_j}}}(-1^{j}))dt\\
\end{split}
\end{equation}